In [28]:
!pip install --upgrade langchain # make sure langchain is installed and up to date
!pip install langchain_community
!pip install pypdf # install the dependency required for PDFLoader
!pip install sentence-transformers
!pip install faiss-cpu
#!pip install transformers

!pip install --upgrade --quiet rapidocr-onnxruntime

In [6]:
from langchain_community.document_loaders import PyPDFLoader

file_path = (
    #"../../docs/integrations/document_loaders/example_data/layout-parser-paper.pdf"
    "https://arxiv.org/pdf/2103.15348.pdf"
)
loader = PyPDFLoader(file_path)
pages = loader.load_and_split()

pages[0]

Document(metadata={'source': 'https://arxiv.org/pdf/2103.15348.pdf', 'page': 0}, page_content='LayoutParser : A Uniﬁed Toolkit for Deep\nLearning Based Document Image Analysis\nZejiang Shen1( \x00), Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain\nLee4, Jacob Carlson3, and Weining Li5\n1Allen Institute for AI\nshannons@allenai.org\n2Brown University\nruochen zhang@brown.edu\n3Harvard University\n{melissadell,jacob carlson }@fas.harvard.edu\n4University of Washington\nbcgl@cs.washington.edu\n5University of Waterloo\nw422li@uwaterloo.ca\nAbstract. Recent advances in document image analysis (DIA) have been\nprimarily driven by the application of neural networks. Ideally, research\noutcomes could be easily deployed in production and extended for further\ninvestigation. However, various factors like loosely organized codebases\nand sophisticated model conﬁgurations complicate the easy reuse of im-\nportant innovations by a wide audience. Though there have been on-going\neﬀorts to im

In [63]:
print(len(pages))
print(type(pages[0]))

16
<class 'langchain_core.documents.base.Document'>


In [51]:
for doc in pages:
  for id, value in doc:
    if id == 'page_content':
      print("VALOR", value)


VALOR LayoutParser : A Uniﬁed Toolkit for Deep
Learning Based Document Image Analysis
Zejiang Shen1(  ), Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain
Lee4, Jacob Carlson3, and Weining Li5
1Allen Institute for AI
shannons@allenai.org
2Brown University
ruochen zhang@brown.edu
3Harvard University
{melissadell,jacob carlson }@fas.harvard.edu
4University of Washington
bcgl@cs.washington.edu
5University of Waterloo
w422li@uwaterloo.ca
Abstract. Recent advances in document image analysis (DIA) have been
primarily driven by the application of neural networks. Ideally, research
outcomes could be easily deployed in production and extended for further
investigation. However, various factors like loosely organized codebases
and sophisticated model conﬁgurations complicate the easy reuse of im-
portant innovations by a wide audience. Though there have been on-going
eﬀorts to improve reusability and simplify deep learning (DL) model
development in disciplines like natural language process

In [77]:
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

import transformers

# Cargar el modelo de embeddings
model_name = "all-MiniLM-L6-v2"
model_kwargs = {'device': 'cpu'}
embeddings = HuggingFaceEmbeddings(model_name=model_name, model_kwargs=model_kwargs)

# Dividir en chunks
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(pages)

# Crear embeddings para cada chunk
document_embeddings = embeddings.embed_documents([t.page_content for t in texts])

pcontent = [t.page_content for t in texts]

text_embedding_pairs = zip(pcontent, document_embeddings)

# Crear un vector database
db = FAISS.from_embeddings(text_embedding_pairs, embeddings)

# Realizar una búsqueda (ejemplo)
#query = "¿Qué dice el documento sobre la inteligencia artificial?"
query = "what the document say about artificial intelligence?"

results = db.similarity_search(query, k=3)

# Imprimir los resultados
for doc in results:
    print("\n",  doc.page_content[:300])


 16 Z. Shen et al.
[23]Paszke, A., Gross, S., Chintala, S., Chanan, G., Yang, E., DeVito, Z., Lin, Z.,
Desmaison, A., Antiga, L., Lerer, A.: Automatic diﬀerentiation in pytorch (2017)
[24]Paszke, A., Gross, S., Massa, F., Lerer, A., Bradbury, J., Chanan, G., Killeen,
T., Lin, Z., Gimelshein, N., Anti

 LayoutParser : A Uniﬁed Toolkit for Deep
Learning Based Document Image Analysis
Zejiang Shen1(  ), Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain
Lee4, Jacob Carlson3, and Weining Li5
1Allen Institute for AI
shannons@allenai.org
2Brown University
ruochen zhang@brown.edu
3Harvard University


 2 Z. Shen et al.
37], layout detection [ 38,22], table detection [ 26], and scene text detection [ 4].
A generalized learning-based framework dramatically reduces the need for the
manual speciﬁcation of complicated rules, which is the status quo with traditional
methods. DL has the potential to tran


In [82]:
from langchain_community.vectorstores import FAISS

query="what the document say about artificial intelligence?"
#query= "What is LayoutParser?"

faiss_index = FAISS.from_documents(pages, embeddings)
docs = faiss_index.similarity_search(query, k=3)
for doc in docs:
    print(str(doc.metadata["page"]) + ":", doc.page_content[:300] + "\n")

15: 16 Z. Shen et al.
[23]Paszke, A., Gross, S., Chintala, S., Chanan, G., Yang, E., DeVito, Z., Lin, Z.,
Desmaison, A., Antiga, L., Lerer, A.: Automatic diﬀerentiation in pytorch (2017)
[24]Paszke, A., Gross, S., Massa, F., Lerer, A., Bradbury, J., Chanan, G., Killeen,
T., Lin, Z., Gimelshein, N., Anti

0: LayoutParser : A Uniﬁed Toolkit for Deep
Learning Based Document Image Analysis
Zejiang Shen1(  ), Ruochen Zhang2, Melissa Dell3, Benjamin Charles Germain
Lee4, Jacob Carlson3, and Weining Li5
1Allen Institute for AI
shannons@allenai.org
2Brown University
ruochen zhang@brown.edu
3Harvard University


1: 2 Z. Shen et al.
37], layout detection [ 38,22], table detection [ 26], and scene text detection [ 4].
A generalized learning-based framework dramatically reduces the need for the
manual speciﬁcation of complicated rules, which is the status quo with traditional
methods. DL has the potential to tran



In [11]:
!ls -l
!mkdir static
!ls -l


total 4
drwxr-xr-x 1 root root 4096 Sep 20 13:22 sample_data
total 8
drwxr-xr-x 1 root root 4096 Sep 20 13:22 sample_data
drwxr-xr-x 2 root root 4096 Sep 24 09:44 static


In [26]:
loader = PyPDFLoader("https://arxiv.org/pdf/2103.15348.pdf", extract_images=True)
pages = loader.load()
pages[4].page_content

'LayoutParser : A Uniﬁed Toolkit for DL-Based DIA 5\nTable 1: Current layout detection models in the LayoutParser model zoo\nDataset Base Model1Large Model Notes\nPubLayNet [38] F / M M Layouts of modern scientiﬁc documents\nPRImA [3] M - Layouts of scanned modern magazines and scientiﬁc reports\nNewspaper [17] F - Layouts of scanned US newspapers from the 20th century\nTableBank [18] F F Table region on modern scientiﬁc and business document\nHJDataset [31] F / M - Layouts of history Japanese documents\n1For each dataset, we train several models of diﬀerent sizes for diﬀerent needs (the trade-oﬀ between accuracy\nvs. computational cost). For “base model” and “large model”, we refer to using the ResNet 50 or ResNet 101\nbackbones [ 13], respectively. One can train models of diﬀerent architectures, like Faster R-CNN [ 28] (F) and Mask\nR-CNN [ 12] (M). For example, an F in the Large Model column indicates it has a Faster R-CNN model trained\nusing the ResNet 101 backbone. The platform i

In [27]:
pages[3].page_content

'4 Z. Shen et al.\nEfficient Data AnnotationC u s t o m i z e d  M o d e l  T r a i n i n gModel Cust omizationDI A Model HubDI A Pipeline SharingCommunity PlatformLa y out Detection ModelsDocument Images \nT h e  C o r e  L a y o u t P a r s e r  L i b r a r yOCR ModuleSt or age & VisualizationLa y out Data Structur e\nFig. 1: The overall architecture of LayoutParser . For an input document image,\nthe core LayoutParser library provides a set of oﬀ-the-shelf tools for layout\ndetection, OCR, visualization, and storage, backed by a carefully designed layout\ndata structure. LayoutParser also supports high level customization via eﬃcient\nlayout annotation and model training functions. These improve model accuracy\non the target samples. The community platform enables the easy sharing of DIA\nmodels and whole digitization pipelines to promote reusability and reproducibility.\nA collection of detailed documentation, tutorials and exemplar projects make\nLayoutParser easy to learn and use